In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot as plt
from pandas import Series
from sklearn.metrics import mean_squared_error
from datetime import datetime
from ipywidgets import IntProgress

/Users/guillermolissa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [2]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from fbprophet import Prophet

In [3]:
from tqdm import tqdm_notebook

In [4]:
import warnings
warnings.filterwarnings('ignore')

### GLOBAL VARIABLES

In [5]:
INPUT_PATH = '../../../data/processed'
INPUT_FILE_NAME = 'dataproc_agg_v01'
OUTPUT_PATH = '../../../models/prophet/hyperparameters'


NRUN = 2
DAYS_PRED = 28
METRIC = 'rmse'

### FUNCTIONS

In [6]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [7]:
# score a model, return None on failure
def score_model(train, cfg, debug=False):
    result = None
    

    m = Prophet(**cfg)


    # show all warnings and fail on exception if debugging
    if debug:
        m.fit(train)

        future = m.make_future_dataframe(periods=DAYS_PRED)

        forecast = m.predict(future)

        y_hat = forecast.iloc[:,-1:]

        y_hat = [i[0] for i in y_hat.values]


    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")

                m.fit(train)
                future = m.make_future_dataframe(periods=DAYS_PRED)

                forecast = m.predict(future)

                y_hat = forecast.iloc[:,-1:]

                y_hat = [i[0] for i in y_hat.values]
        except:
            y_hat = None
    
        
        
    return y_hat

### LOAD DATASET

data = pd.read_pickle(f'{INPUT_PATH}/{INPUT_FILE_NAME}.pkl')

In [8]:
data = pd.read_pickle(f'{INPUT_PATH}/{INPUT_FILE_NAME}.pkl')

In [10]:
date_cutoff = data.d.max() - DAYS_PRED

X_train = data[data.d <= date_cutoff]

X_test = data[data.d > date_cutoff]

del data

In [12]:
features = ['state_id','date', 'demand_mean']
y_true = 'demand_mean'
level = 'state_id'

In [13]:
X_train = X_train[features]
X_test = X_test[features]

## TRAIN MODEL

In [14]:
cfg_prophet = dict()

In [15]:
cfg_prophet['yearly_seasonality'] = True
cfg_prophet['daily_seasonality'] = False
cfg_prophet['seasonality_prior_scale'] = 0.1

In [16]:
errors = []
predictions = []
idxs = []

for idx in tqdm_notebook(X_train[level].unique()):
    train = X_train[X_train[level] == idx]
    test = X_test[X_test[level] == idx][y_true].tolist()
    idxs.append(idx) # save to make a new dataframe with y_hat

    train.drop(level, axis=1, inplace=True)
    train.columns = ['ds', 'y']
    
    y_hat = score_model(train, cfg_prophet, debug=False)
    predictions.append(y_hat)
    
    errors.append(rmse(test,y_hat[-DAYS_PRED:]))


In [17]:
errors

[1.480046162529873, 1.2263932638331798, 1.382101200135887]

In [ ]:
predictions = np.array(predictions)

In [ ]:
np.save(f'{OUTPUT_PATH}/predictions.npy', predictions)

In [ ]:
np.save(f'{OUTPUT_PATH}/idxs.npy', idxs)

In [ ]:
np.save(f'{OUTPUT_PATH}/errors.npy', errors)